In [1]:
import os
import numpy as np
import pickle as pk
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Dropout, Flatten, Activation
import cv2
from string import ascii_uppercase
import operator

In [ ]:
def make_data(mode):
    
    org_path = os.getcwd()
    basepath = org_path + '\\data\\' + mode
    img_data = []
    target = []

    for letter in os.listdir(basepath):

        images_path = basepath + '\\' + letter
       
        for image in os.listdir(images_path):
            
            img_path = images_path + '\\' + image
            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            test_image = cv2.resize(gray, (128,128))
            test_image = test_image.reshape(128, 128, 1)
            img_data.append(test_image)
                
            if(letter == '0'):
                target.append(0)
            else :
                target.append(ord(letter) - ord('A') + 1)
                
    m = len(img_data)
    nh = img_data[0].shape[0]
    nw = img_data[0].shape[1]
    nc = img_data[0].shape[2]

    X_t = np.zeros((m, nh, nw, nc))

    for i in range(m):
        for j in range(nh):
            for k in range(nw):
                for l in range(nc):
                    X_t[i][j][k][l] = img_data[i][j][k][l]
    
    X_t = X_t/255
    Y_t = np.asarray(target).reshape(-1, 1)
    
    return X_t, Y_t

In [ ]:
#X, Y = make_data('train')
f = open('training_data.pkl', 'rb')
X = pk.load(f)
f.close()
f = open('targets.pkl', 'rb')
Y = pk.load(f)
f.close()

'''f = open('training_data.pkl', 'wb')
pk.dump(X, f)
f.close()
f = open('targets.pkl', 'wb')
pk.dump(Y, f)
f.close()'''

In [ ]:
#X_test, Y_test = make_data('test')

In [ ]:
def make_model():
    
    classifier = Sequential()
    sz = 128

    classifier.add(Conv2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Conv2D(32, (3, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    classifier.add(Flatten())
    classifier.add(Dense(units=128, activation='relu'))
    classifier.add(Dropout(0.40))
    classifier.add(Dense(units=96, activation='relu'))
    classifier.add(Dropout(0.40))
    classifier.add(Dense(units=64, activation='relu'))
    classifier.add(Dense(units=27, activation='softmax'))
    
    return classifier

In [ ]:
'''model = make_model()
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(X, Y, epochs = 3, batch_size = 32)'''

In [2]:
def load_my_model(models_path, model_name):
    
    #os.chdir(models_path)
    json = models_path + model_name + '.json'
    json_file = open(json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    weights = models_path + model_name + '.h5'
    model.load_weights(weights)
    
    return model

In [3]:
basepath = os.getcwd()
models_path = basepath + '\\model\\'

model = load_my_model(models_path, 'model-bw')
model_dru = load_my_model(models_path, 'model-bw_dru')
model_smn = load_my_model(models_path, 'model-bw_smn')
model_tkdi = load_my_model(models_path, 'model-bw_tkdi')

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)             

In [13]:
def prediction(test_image):
    
    
    test_image = cv2.resize(test_image, (128,128))
    test_image = test_image.reshape(1, 128, 128, 1)
    result = model.predict(test_image)
    result_dru = model_dru.predict(test_image)
    result_tkdi = model_tkdi.predict(test_image)
    result_smn = model_smn.predict(test_image)
    prediction={}
    prediction['blank'] = result[0][0]
    inde = 1
    current_symbol = ''
    
    for i in ascii_uppercase:
        prediction[i] = result[0][inde]
        inde += 1
        
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    current_symbol = prediction[0][0]

    if(current_symbol == 'D' or current_symbol == 'R' or current_symbol == 'U'):
        prediction = {}
        prediction['D'] = result_dru[0][0]
        prediction['R'] = result_dru[0][1]
        prediction['U'] = result_dru[0][2]
        prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
        current_symbol = prediction[0][0]

    if(current_symbol == 'D' or current_symbol == 'I' or current_symbol == 'K' or current_symbol == 'T'):
        prediction = {}
        prediction['D'] = result_tkdi[0][0]
        prediction['I'] = result_tkdi[0][1]
        prediction['K'] = result_tkdi[0][2]
        prediction['T'] = result_tkdi[0][3]
        prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
        current_symbol = prediction[0][0]

    if(current_symbol == 'M' or current_symbol == 'N' or current_symbol == 'S'):
        prediction1 = {}
        prediction1['M'] = result_smn[0][0]
        prediction1['N'] = result_smn[0][1]
        prediction1['S'] = result_smn[0][2]
        prediction1 = sorted(prediction1.items(), key=operator.itemgetter(1), reverse=True)
        if(prediction1[0][0] == 'S'):
            current_symbol = prediction1[0][0]
        else:
            current_symbol = prediction[0][0]
            
    if(current_symbol == 'blank'):
            return ""
        
    else:
        return current_symbol